In [1]:
import json
import requests
import pandas as pd
import time
import datetime

In [2]:
lol_api_key = 'RGAPI-25bc6d6f-3db1-4a0d-a08a-6823a456d5d5'

In [3]:
URLS = {'challenge' : 'https://kr.api.riotgames.com/lol/league/v3/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'master' : 'https://kr.api.riotgames.com/lol/league/v3/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=',
        'summoners' : 'https://kr.api.riotgames.com/lol/summoner/v3/summoners/',
        'matches' : 'https://kr.api.riotgames.com/lol/match/v3/matchlists/by-account/'
       }

In [4]:
# challenge league 선수들 정보 불러옴
url = requests.get(URLS['challenge']+lol_api_key)
print(url.status_code)
challenge = url.json()

200


In [5]:
# master league 선수들 정보 불러옴
url = requests.get(URLS['master']+lol_api_key)
print(url.status_code)
master = url.json()

200


In [6]:
# match를 모을 target을 challenge, master 리그 summerner들로 잡음
df_target_origin = pd.DataFrame(challenge['entries']).append(pd.DataFrame(master['entries']), ignore_index=True)
df_target_origin[df_target_origin.playerOrTeamName == 'Hide on bush']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins
182,False,True,False,1215,795,4460427,Hide on bush,I,True,892


In [ ]:
# X-App-Rate-Limit에 맞게 시간을 쉬면서 돌려줌 status_code=429 는 api호출 한계 초과임
summoners=[]
counts=0
for i in df_target.playerOrTeamId:
    time.sleep(0.84)
#     counts += 1
#     print(counts)
    url = requests.get(URLS['summoners'] + i + '?api_key=' +lol_api_key)
    while url.status_code == 429:
        time.sleep(1)
        url = requests.get(URLS['summoners'] + i + '?api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
    else:
        summoners.append(url.json())

pd.DataFrame(summoners).to_csv('summonors.csv', index = False)

In [7]:
df_summonors = pd.read_csv('summonors.csv')[['accountId','id']]

In [8]:
# 이렇게 안하면 merge시 조인이 안됨 <- 데이터 타입이 달라서 생기는 문제인데 꼭 이렇게 해야 할까?
df_target_origin['playerOrTeamId'] = df_target_origin['playerOrTeamId'].astype(int)

In [9]:
df_target = pd.merge(df_target_origin, df_summonors, left_on='playerOrTeamId', right_on = 'id', 
                     how='inner')

In [10]:
df_target[df_target.playerOrTeamName == 'Hide on bush']

,freshBlood,hotStreak,inactive,leaguePoints,losses,playerOrTeamId,playerOrTeamName,rank,veteran,wins,accountId,id
182,False,True,False,1215,795,4460427,Hide on bush,I,True,892,3440481,4460427


# match

In [11]:
# version별 분석을 하고 싶어서 기준일을 timestamp로 지정한다.
ver7_19_timestamp_begin = int(datetime.datetime.timestamp(datetime.datetime(2017,9,28,0,0,0))) * 1000
ver7_19_timestamp_end = int(datetime.datetime.timestamp(datetime.datetime(2017,10,3,0,0,0))) * 1000

print(ver7_19_timestamp_begin, ver7_19_timestamp_end)

1506524400000 1506956400000


In [12]:
# match 를 선수별로 모은다.
# 선수당 max 100개만 모은다. 라이엇 정책이 1회 조회에 100개. / 100개 이상으로도 돌릴 수 있을 것 같긴 한데 이번 것에선 여기까지.
matches = []
counts = 0
for i in df_target.accountId:
#     time.sleep(0.84)
    counts += 1
    print(counts, i)
    url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(ver7_19_timestamp_end) + '&beginTime=' + \
                       str(ver7_19_timestamp_begin) + '&api_key=' + lol_api_key)
    while url.status_code == 429:
        time.sleep(1)
        url = requests.get(URLS['matches'] + str(i) + '?endTime=' + str(ver7_19_timestamp_end) + '&beginTime=' + \
                           str(ver7_19_timestamp_begin) + '&api_key=' + lol_api_key)
    if url.status_code != 200:
        print(i, url.status_code)
    else:
        matches.append(url.json())


1 200621317
200621317 404
2 204333883
3 1805320
4 209319733
5 1198324
6 207617201
7 203492844
8 209376444
9 207443723
10 207151221
11 206487982
12 207151225
13 209300003
14 208481201
15 209430043
16 204245747
17 204531622
18 2461308
19 209455940
209455940 404
20 203769662


KeyboardInterrupt: 

# 참고자료

In [ ]:
API_KEY = os.environ['LOL_API_KEY']
URLS = {
        'NAME_TO_ID' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-name/{name}',
        'ID_TO_NAME' : 'https://na1.api.riotgames.com/lol/summoner/v3/summoners/by-account/{id}',
        'MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}?season=8',
        'RECENT_MATCHES_BY_ID' : 'https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/{id}/recent',
        'MATCH_INFO': 'https://na1.api.riotgames.com/lol/match/v3/matches/{id}', 
        'RECENT_TEN' : 'https://na.api.riotgames.com/api/lol/NA/v1.3/game/by-summoner/{summonerId}/recent',
        }
HEADERS = {'X-Riot-Token' : API_KEY}

CHAMP_BY_ID = {}
with open('data/champ_by_id.json') as f:
    CHAMP_BY_ID = json.load(f)

CHAMP_BY_NAME = {}
with open('data/champ_by_name.json') as f:
    CHAMP_BY_NAME = json.load(f)


print(CHAMP_BY_ID['data']['236'])

In [ ]:
from apiKeys import API_KEY
import requests
import sys
import pprint

USERNAME = sys.argv[1]
BASE_URL = "https://na1.api.riotgames.com/lol/"
SUMMONER_ID_URL = BASE_URL + "summoner/v3/summoners/by-name/"
MASTERY_URL = BASE_URL + "platform/v3/masteries/by-summoner/"

r = requests.get(SUMMONER_ID_URL + USERNAME + "?api_key=" + API_KEY)
print r.status_code
data = r.json()
print data
print data["name"]
print data["id"]
summoner_id = data["id"]
r = requests.get(MASTERY_URL + str(summoner_id) + "?api_key=" + API_KEY)
pprint.pprint(r.json())
